# 3.0 - Modelado, ajuste y evaluación

In [1]:
# librerias

import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)

from catboost import CatBoostRegressor as CTR

from sklearn.model_selection import train_test_split as tts 

from hyperopt import fmin, hp, tpe, Trials, space_eval, STATUS_OK

from sklearn.metrics import mean_squared_error as mse 
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import r2_score as r2

**Datos**

Selecciono la características previamente evaluadas y solamente cargo esas. Elimino los outliers y cambio el tipo de dato.

In [2]:
cols=['accommodates', 'air_conditioning', 'availability_30', 'availability_365', 'availability_60', 'availability_90',
      'bathrooms', 'bedrooms', 'beds', 'calculated_host_listings_count', 'calculated_host_listings_count_entire_homes',
      'calculated_host_listings_count_private_rooms', 'calculated_host_listings_count_shared_rooms', 'cleaning_fee',
      'dishwasher', 'extra_people', 'guests_included','latitude', 'longitude', 'maximum_nights', 'minimum_nights', 
      'number_of_reviews', 'number_of_reviews_ltm', 'room_type_private_room', 'room_type_shared_room', 'security_deposit',
      'price']

len(cols)

27

In [3]:
# carga de datos
listings=pd.read_csv('../data/transform_data/listings_normal.csv', usecols=cols)

listings=listings[(listings.price>=10) & (listings.price<=196)]  # eliminacion de outliers

# cambio en el tamaño del tipo de dato
for c in listings.select_dtypes(include='int'):
    listings[c]=pd.to_numeric(listings[c], downcast='integer')

for c in listings.select_dtypes(include='float'):
    listings[c]=pd.to_numeric(listings[c], downcast='float')

**Modelo**

Importo y entreno un Catboost Regressor, realizo predicción con el paquete de entrenamiento y de testeo, para darme cuenta de posible sobreajuste, midiendo RMSE, MAE y R2.

In [4]:
ctr=CTR(verbose=0)

In [5]:
X=listings.drop('price', axis=1)
y=listings.price

X_train, X_test, y_train, y_test = tts(X, y, train_size=0.8, test_size=0.2, random_state=42)

ctr.fit(X_train, y_train)

y_pred=ctr.predict(X_train)

print(f'Train RMSE: {mse(y_train, y_pred, squared=False)}')
print(f'Train MAE: {mae(y_train, y_pred)}')
print(f'Train R2: {r2(y_train, y_pred)}')

Train RMSE: 17.09389499575402
Train MAE: 11.909514534360929
Train R2: 0.7995457095749583


In [6]:
y_pred=ctr.predict(X_test)

print(f'Test RMSE: {mse(y_test, y_pred, squared=False)}')
print(f'Test MAE: {mae(y_test, y_pred)}')
print(f'Test R2: {r2(y_test, y_pred)}')

Test RMSE: 21.713858907632996
Test MAE: 14.82603810439655
Test R2: 0.7113638913656304


**Ajuste de hiperparámetros**

Ajuste bayesiano de hiperparámetros.

In [7]:
# aprioris, espacio de hiperparámetros

espacio={
    'n_estimators':hp.quniform('n_estimators', 100, 700, 25),
    
    'learning_rate':hp.uniform('learning_rate', 0.01, 1.0),
    
    'depth':hp.quniform('depth', 7, 16, 1)
}

In [8]:
def objetivo(espacio):
    
    modelo=CTR(verbose=0,
               n_estimators=int(espacio['n_estimators']),
               learning_rate=espacio['learning_rate'],
               depth=int(espacio['depth'])
              )
    
    eval_set=[(X_train, y_train), (X_test, y_test)]
    
    modelo.fit(X_train, y_train, eval_set=eval_set)
    
    y_pred=modelo.predict(X_test)
    
    rmse=mse(y_test, y_pred, squared=False)
    
    return {'loss': rmse, 'status': STATUS_OK}

In [9]:
mejor=fmin(fn=objetivo,
          space=espacio,
          algo=tpe.suggest,
          max_evals=20,
          trials=Trials())

mejor

100%|████████████████████████████████████████████████████| 20/20 [03:45<00:00, 11.28s/trial, best loss: 21.908005638155686]


{'depth': 10.0, 'learning_rate': 0.11517739200904813, 'n_estimators': 250.0}

In [10]:
modelo_ajustado=CTR(
    verbose=0,
    n_estimators=int(mejor['n_estimators']),
    learning_rate=mejor['learning_rate'],
    depth=int(mejor['depth']),
    )


modelo_ajustado.fit(X_train, y_train)

In [11]:
y_pred=modelo_ajustado.predict(X_train)

print(f'Train RMSE: {mse(y_train, y_pred, squared=False)}')
print(f'Train MAE: {mae(y_train, y_pred)}')
print(f'Train R2: {r2(y_train, y_pred)}')

Train RMSE: 14.01635828564621
Train MAE: 9.655967376858882
Train R2: 0.8652267587326375


In [12]:
y_pred=modelo_ajustado.predict(X_test)

print(f'Test RMSE: {mse(y_test, y_pred, squared=False)}')
print(f'Test MAE: {mae(y_test, y_pred)}')
print(f'Test R2: {r2(y_test, y_pred)}')

Test RMSE: 21.925796297528173
Test MAE: 14.946124581480271
Test R2: 0.7057019478848928


**Evaluación**

En todo momento estoy usando el MAE, RMSE y R2. Veamos su definición y su explicación.

###### MAE (Error Medio Absoluto)


$$MAE = \frac{1}{n}\sum_{i=1}^{n}|y_i-\hat{y}_i|$$


pertenece al intervalo [0, +$\infty$)

###### RMSE


$$RMSE = \sqrt{\frac{1}{n}\sum_{i=1}^{n}(y_i-\hat{y}_i)^{2}}$$


pertenece al intervalo [0, +$\infty$) y se cumple que:

$$MAE <= RMSE <= MAE · \sqrt{n}$$

Ambas métricas son indiferentes al signo del error, pero el RMSE, al hacer la media de cuadrados, es sensible a errores más grandes, y está en ese intervalo con respector al MAE. Grandes diferencias entre MAE y RMSE indican la existencia de valores extremos (outliers), de ahí usar ambas métricas. También, ambas métricas tienen las unidades de la variable a predecir, por lo que son más fáciles de leer que, por ejemplo, el MSE.

###### R2


$$R2 = 1 - \frac{\sum_{i=1}^{n}(y_i-\hat{y}_i)^{2}}{\sum_{i=1}^{n}(y_i-\bar{y})^{2}}$$

pertenece al intervalo (-$\infty$, 1]

En cuanto al R2, nos da una tasa entre error y varianza. Básicamente nos indica cuánta varianza del objetivo somos capaces de explicar según nuestros predictores.

**Hiperparámetros:** Se opta por el modelo Catboost con sus valores por defecto, dado que el ajuste de los hiperparámetros me genera un ligero sobreajuste (overfitting).